In [2]:
!dlt --version

/Volumes/Sozhan-External/Repository/llm-zoomcamp-2025/llmenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
dlt 1.13.0


In [3]:
import dlt
import requests

@dlt.resource(write_disposition="replace", name="zoomcamp_data")
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

/Volumes/Sozhan-External/Repository/llm-zoomcamp-2025/llmenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="database.qdrant", 
)

In [5]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Run started at 2025-07-12 12:53:33.815744+00:00 and COMPLETED in 27.74 seconds with 4 steps.
Step extract COMPLETED in 2.02 seconds.

Load package 1752324834.999309 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.05 seconds.
Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- zoomcamp_data: 948 row(s)

Load package 1752324834.999309 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 4.49 seconds.
Pipeline zoomcamp_pipeline load step completed in 4.48 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /Volumes/Sozhan-External/Repository/llm-zoomcamp-2025/dlt-workshop/database.qdrant location to store data
Load package 1752324834.999309 is LOADED and contains no failed jobs

Step run COMPLETED in 27.74 seconds.
Pipeline zoomcamp_pipeline load step completed in 4.48 seconds
1 lo

In [7]:
import json
from pprint import pprint

with open(r"database.qdrant/meta.json", 'r', encoding="utf-8") as jsonfile:
    content = json.load(jsonfile)
    pprint(content)

{'aliases': {},
 'collections': {'zoomcamp_tagged_data': {'hnsw_config': None,
                                          'init_from': None,
                                          'on_disk_payload': None,
                                          'optimizers_config': None,
                                          'quantization_config': None,
                                          'replication_factor': None,
                                          'shard_number': None,
                                          'sharding_method': None,
                                          'sparse_vectors': None,
                                          'strict_mode_config': None,
                                          'vectors': {'fast-bge-small-en': {'datatype': None,
                                                                            'distance': 'Cosine',
                                                                            'hnsw_config': None,
                           